In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import time
import joblib

In [51]:
# Список моделей
models = {
    # Логистическая регрессия: 
    # - Простой и интерпретируемый метод
    # - Хорошо работает с бинарной классификацией
    # - Эффективен для линейно разделимых данных
    'Logistic Regression': LogisticRegression(max_iter=10000),
    # Дерево решений:
    # - Интуитивно понятен и легко интерпретируется
    # - Может моделировать нелинейные зависимости
    # - Хорошо подходит для смешанных типов данных
    'Decision Tree': DecisionTreeClassifier(max_depth=5),
    # k-ближайших соседей:
    # - Простой и не требует предположений о распределении данных
    # - Эффективен для задач с локальной структурой данных
    # - Подходит для небольших и средних датасетов
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    # Наивный Байес:
    # - Очень быстрый и требует мало данных для обучения
    # - Хорошо работает с независимыми признаками
    # - Прост в реализации и эффективен для базовой классификации
    'Naive Bayes': GaussianNB()
}

# Словарь для хранения результатов всех моделей по всем датасетам
all_results = {name: {'accuracy': [], 'time': []} for name in models.keys()}

In [52]:
# Обработка всех 12 датасетов
for i in range(1, 13):
    # Загрузка датасета
    dataset_name = f'dataset_{i}.csv'

    df = pd.read_csv(dataset_name)
    
    # Подготовка данных
    le_figure1 = LabelEncoder()
    le_figure2 = LabelEncoder()
    df['figure1'] = le_figure1.fit_transform(df['figure1'])
    df['figure2'] = le_figure2.fit_transform(df['figure2'])
    
    if 'color1' in df.columns:
        le_color1 = LabelEncoder()
        le_color2 = LabelEncoder()
        df['color1'] = le_color1.fit_transform(df['color1'])
        df['color2'] = le_color2.fit_transform(df['color2'])
    
    # Разделение данных
    X = df.drop('collision', axis=1)
    y = df['collision']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Обучение и оценка моделей
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time
        accuracy = model.score(X_test, y_test)
        
        # Сохранение результатов
        all_results[name]['accuracy'].append(accuracy)
        all_results[name]['time'].append(training_time)

In [53]:
# Анализ результатов
summary = {}
for name in models.keys():
    avg_accuracy = sum(all_results[name]['accuracy']) / len(all_results[name]['accuracy'])
    avg_time = sum(all_results[name]['time']) / len(all_results[name]['time'])
    summary[name] = {'avg_accuracy': avg_accuracy, 'avg_time': avg_time}

sorted_models = sorted(summary.items(), key=lambda x: x[1]['avg_accuracy'], reverse=True)

In [54]:
# Вывод двух лучших моделей
print("Две лучшие модели:")
for i, (name, metrics) in enumerate(sorted_models[:2], 1):
    print(f"{i}. {name}")
    print(f"   Средняя точность: {metrics['avg_accuracy']:.4f}")
    print(f"   Среднее время обучения: {metrics['avg_time']:.4f} сек")
    filename = f'best_model_{i}_{name.lower().replace(" ", "_")}.pkl'
    joblib.dump(model, filename)
    print(f"Сохранена модель {name} как {filename}")
    



Две лучшие модели:
1. Decision Tree
   Средняя точность: 0.6687
   Среднее время обучения: 0.0036 сек
Сохранена модель Decision Tree как best_model_1_decision_tree.pkl
2. Naive Bayes
   Средняя точность: 0.6312
   Среднее время обучения: 0.0022 сек
Сохранена модель Naive Bayes как best_model_2_naive_bayes.pkl
